In [22]:
# Import findspark and initialize.
import findspark
findspark.init()

In [23]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [24]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [25]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [26]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query1 = """
(SELECT
YEAR(date) AS YEAR,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC)
"""
spark.sql(query1).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [27]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query2 = """
(SELECT
YEAR(date_built) AS YEAR,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND
bathrooms = 3
GROUP BY YEAR
ORDER BY YEAR DESC)
"""
spark.sql(query2).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [28]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query3 = """
(SELECT
YEAR(date_built) AS YEAR,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND
bathrooms = 3 AND
floors = 2 AND
sqft_living >= 2000
GROUP BY YEAR
ORDER BY YEAR DESC)
"""
spark.sql(query3).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    280317.58|
|2016|     293965.1|
|2015|    297609.97|
|2014|    298264.72|
|2013|    303676.79|
|2012|    307539.97|
|2011|    276553.81|
|2010|    285010.22|
+----+-------------+



In [29]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = """
(SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC)
"""
spark.sql(query4).show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.6671593189239502 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query5 = """
(SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC)
"""
spark.sql(query5).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.6539962291717529 seconds ---


In [33]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')

In [34]:
# Compress the 'p_home_sales' directory into a zip file named 'p_home_sales.zip'
!zip -r p_home_sales.zip p_home_sales

# Download the 'p_home_sales.zip' file to the local machine
from google.colab import files
files.download('p_home_sales.zip')

updating: p_home_sales/ (stored 0%)
updating: p_home_sales/date_built=2013/ (stored 0%)
updating: p_home_sales/._SUCCESS.crc (stored 0%)
updating: p_home_sales/date_built=2015/ (stored 0%)
updating: p_home_sales/date_built=2012/ (stored 0%)
updating: p_home_sales/date_built=2011/ (stored 0%)
updating: p_home_sales/date_built=2014/ (stored 0%)
updating: p_home_sales/date_built=2016/ (stored 0%)
updating: p_home_sales/date_built=2010/ (stored 0%)
updating: p_home_sales/date_built=2017/ (stored 0%)
updating: p_home_sales/_SUCCESS (stored 0%)
  adding: p_home_sales/date_built=2013/part-00000-e8033976-4b39-417e-a4dd-61f0b39e37bc.c000.snappy.parquet (deflated 20%)
  adding: p_home_sales/date_built=2013/.part-00000-e8033976-4b39-417e-a4dd-61f0b39e37bc.c000.snappy.parquet.crc (stored 0%)
  adding: p_home_sales/date_built=2015/part-00000-e8033976-4b39-417e-a4dd-61f0b39e37bc.c000.snappy.parquet (deflated 20%)
  adding: p_home_sales/date_built=2015/.part-00000-e8033976-4b39-417e-a4dd-61f0b39e37bc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('p_home_sales')

In [36]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('parquet_temp_home')

In [37]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query6 = """
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM parquet_temp_home
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(query6).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.5044565200805664 seconds ---


In [38]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [39]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('home_sales is not cached')


home_sales is not cached
